In [ ]:
!pip install rdkit-pypi

In [ ]:
!curl -LO  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/')

!conda install -y -c rdkit rdkit 

In [ ]:
!curl -o ZINC.smiles https://raw.githubusercontent.com/heartcored98/Standalone-DeepLearning/master/Lec9/ZINC.smiles
!curl -o vocab.npy https://raw.githubusercontent.com/heartcored98/Standalone-DeepLearning/master/Lec9/vocab.np

In [ ]:
import argparse
import sys
import time
import copy

import numpy as np

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Crippen import MolLogP

from sklearn.metrics import mean_absolute_error 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from tqdm import tnrange, tqdm_notebook
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(123)
torch.manual_seed(123)

torch.set_default_tensor_type('torch.FloatTensor')

In [ ]:
def read_ZINC_smiles(file_name, num_mol):
    f = open(file_name, 'r')
    contents = f.readlines()

    smi_list = [] # 분자의 그래프 input
    logP_list = [] # 화학적 특성 

    for i in tqdm_notebook(range(num_mol), desc='Reading Data'):
        smi = contents[i].strip() # 파일에 데이터를 저장할때 공백부분을 제거해서 smi 저장
        m = Chem.MolFromSmiles(smi)
        smi_list.append(smi)
        logP_list.append(MolLogP(m))

    logP_list = np.asarray(logP_list).astype(float)

    return smi_list, logP_list


def smiles_to_onehot(smi_list):
    def smiles_to_vector(smiles, vocab, max_length):
        while len(smiles) < max_length:
            smiles += " "
        vector = [vocab.index(str(x)) for x in smiles]
        one_hot = np.zeros((len(vocab), max_length), dtype=int)
        for i, elm in enumerate(vector):
            one_hot[elm][i] = 1
        return one_hot

    vocab = np.load('./vocab.npy')
    smi_total = []

    for i, smi in tqdm_notebook(enumerate(smi_list), desc='Converting to One Hot'):
        smi_onehot = smiles_to_vector(smi, list(vocab), 120)
        smi_total.append(smi_onehot)

    return np.asarray(smi_total)

def convert_to_graph(smiles_list):
    adj = []
    adj_norm = []
    features = []
    maxNumAtoms = 50 # 최대 원자가 50개
    for i in tqdm_notebook(smiles_list, desc='Converting to Graph'):
        # Mol
        iMol = Chem.MolFromSmiles(i.strip())
        #Adj
        iAdjTmp = Chem.rdmolops.GetAdjacencyMatrix(iMol)
        # Feature
        if( iAdjTmp.shape[0] <= maxNumAtoms):
            # Feature-preprocessing
            iFeature = np.zeros((maxNumAtoms, 58))
            iFeatureTmp = []
            for atom in iMol.GetAtoms():
                iFeatureTmp.append( atom_feature(atom) ) ### atom features only
            iFeature[0:len(iFeatureTmp), 0:58] = iFeatureTmp ### 0 padding for feature-set
            features.append(iFeature)

            # Adj-preprocessing
            iAdj = np.zeros((maxNumAtoms, maxNumAtoms))
            iAdj[0:len(iFeatureTmp), 0:len(iFeatureTmp)] = iAdjTmp + np.eye(len(iFeatureTmp))
            adj.append(np.asarray(iAdj))
    features = np.asarray(features)

    return features, adj
    
def atom_feature(atom):
    return np.array(one_of_k_encoding_unk(atom.GetSymbol(),
                                      ['C', 'N', 'O', 'S', 'F', 'H', 'Si', 'P', 'Cl', 'Br',
                                       'Li', 'Na', 'K', 'Mg', 'Ca', 'Fe', 'As', 'Al', 'I', 'B',
                                       'V', 'Tl', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn',
                                       'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'Mn', 'Cr', 'Pt', 'Hg', 'Pb']) +
                    one_of_k_encoding(atom.GetDegree(), [0, 1, 2, 3, 4, 5]) +
                    one_of_k_encoding_unk(atom.GetTotalNumHs(), [0, 1, 2, 3, 4]) +
                    one_of_k_encoding_unk(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5]) +
                    [atom.GetIsAromatic()])    # (40, 6, 5, 6, 1)

def one_of_k_encoding(x, allowable_set):
    if x not in allowable_set:
        raise Exception("input {0} not in allowable set{1}:".format(x, allowable_set))
    return list(map(lambda s: x == s, allowable_set))

def one_of_k_encoding_unk(x, allowable_set):
    """Maps inputs not in the allowable set to the last element."""
    if x not in allowable_set:
        x = allowable_set[-1]
    return list(map(lambda s: x == s, allowable_set))

In [ ]:
list_smi, list_logP = read_ZINC_smiles('ZINC.smiles', 23040)
list_feature, list_adj = convert_to_graph(list_smi)

In [ ]:
class GCNDataset(Dataset):
    def __init__(self, list_feature, list_adj, list_logP):
        self.list_feature = list_feature
        self.list_adj = list_adj
        self.list_logP = list_logP

    def __len__(self):
        return len(self.list_feature)

    def __getitem__(self, index):
        return self.list_feature[index], self.list_adj[index], self.list_logP[index]


def partition(list_feature, list_adj, list_logP):
    test_size = 0.2
    val_size = 0.2
    num_total = list_feature.shape[0]
    num_train = int(num_total * (1 - test_size - val_size))
    num_val = int(num_total * val_size)
    num_test = int(num_total * test_size)

    feature_train = list_feature[:num_train]
    adj_train = list_adj[:num_train]
    logP_train = list_logP[:num_train]
    feature_val = list_feature[num_train:num_train + num_val]
    adj_val = list_adj[num_train:num_train + num_val]
    logP_val = list_logP[num_train:num_train + num_val]
    feature_test = list_feature[num_total - num_test:]
    adj_test = list_adj[num_total - num_test:]
    logP_test = list_logP[num_total - num_test:]
        
    train_set = GCNDataset(feature_train, adj_train, logP_train)
    val_set = GCNDataset(feature_val, adj_val, logP_val)
    test_set = GCNDataset(feature_test, adj_test, logP_test)

    partition = {
        'train': train_set,
        'val': val_set,
        'test': test_set
    }

    return partition

In [ ]:
partition = partition(list_feature, list_adj, list_logP)

In [ ]:
class GCNLayer(nn.Module):
    
    def __init__(self, in_dim, out_dim, n_atom, act=None, bn=False):        super(GCNLayer, self).__init__()
        
        self.use_bn = bn
        self.linear = nn.Linear(in_dim, out_dim)        nn.init.xavier_uniform_(self.linear.weight)
        self.bn = nn.BatchNorm1d(n_atom)
        self.activation = act
        
    def forward(self, x, adj):
        out = self.linear(x) #HW + b
        out = torch.matmul(adj, out) #AHW + b
        if self.use_bn:
            out = self.bn(out)
        if self.activation != None:
            out = self.activation(out)
        return out, adj

In [ ]:
class Predictor(nn.Module):
    
    def __init__(self, in_dim, out_dim, act=None):
        super(Predictor, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.linear = nn.Linear(self.in_dim,
                                self.out_dim)
        nn.init.xavier_uniform_(self.linear.weight)
        self.activation = act
        
    def forward(self, x):
        out = self.linear(x)
        if self.activation != None:
            out = self.activation(out)
        return out

In [ ]:
class ReadOut(nn.Module):
    
    def __init__(self, in_dim, out_dim, act=None):
        super(ReadOut, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim= out_dim
        
        self.linear = nn.Linear(self.in_dim, 
                                self.out_dim)
        nn.init.xavier_uniform_(self.linear.weight)
        self.activation = act

    def forward(self, x):
        out = self.linear(x)
        out = torch.sum(out, 1)
        if self.activation != None:
            out = self.activation(out)
        return out

In [ ]:
class GCNNet(nn.Module):
    
    def __init__(self):
        super(GCNNet, self).__init__()
        out_dim2 = 128
        hidden_dim = out_dim2
        pred_dim1 = 128
        pred_dim2 = 64
        pred_dim3 = 32

        self.gcn_layer1 = GCNLayer(in_dim = 58, out_dim = 116, n_atom = 50, bn=True)
        self.gcn_layer2 = GCNLayer(in_dim = 116, out_dim = 232, n_atom = 50, bn=True)
        self.gcn_layer3 = GCNLayer(in_dim = 232, out_dim = out_dim2, n_atom = 50, bn=True)

        self.relu = nn.ReLU()

        self.readout = ReadOut(hidden_dim, 
                               pred_dim1,
                               act=nn.ReLU())
        self.pred1 = Predictor(pred_dim1,
                               pred_dim2,
                               act=nn.ReLU())
        self.pred2 = Predictor(pred_dim2,
                               pred_dim3,
                               act=nn.ReLU())
        self.pred3 = Predictor(pred_dim3,
                               1)
        
    def forward(self, x, adj):
        out, adj = self.gcn_layer1(x,adj)
        out = self.relu(out)
        out, adj = self.gcn_layer2(out, adj)
        out = self.relu(out)
        out, adj = self.gcn_layer3(out, adj)
        out = self.relu(out)
        out = self.readout(out)
        out = self.pred1(out)
        out = self.pred2(out)
        out = self.pred3(out)
        return out

In [ ]:
def train(net, partition, optimizer, criterion):
    train_batch_size = 256
    trainloader = torch.utils.data.DataLoader(partition['train'], 
                                              batch_size=train_batch_size, 
                                              shuffle=False, num_workers=2)
    net.train()

    train_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad()

        # get the inputs
        list_feature, list_adj, list_logP = data

        list_feature = list_feature.float()
        list_adj = list_adj.float()
        list_logP = list_logP.float().view(-1, 1)
        outputs = net(list_feature, list_adj)

        loss = criterion(outputs, list_logP)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    train_loss = train_loss / len(trainloader)
    return train_loss

In [ ]:
def validate(net, partition, criterion):
    val_batch_size = 128
    validloader = torch.utils.data.DataLoader(partition['val'], 
                                              batch_size=val_batch_size, 
                                              shuffle=False, num_workers=2)

    net.eval()

    val_loss = 0.0
    with torch.no_grad():
        for data in validloader:
            list_feature, list_adj, list_logP = data

            list_feature = list_feature.float()
            list_adj = list_adj.float()
            list_logP = list_logP.float().view(-1, 1)
            outputs = net(list_feature, list_adj)

            loss = criterion(outputs, list_logP)
            val_loss += loss.item()

        val_loss = val_loss / len(validloader)
    return val_loss

In [ ]:
def test(net, partition):
    test_batch_size = 128
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=test_batch_size, 
                                             shuffle=False, num_workers=2)
    net.eval()
    with torch.no_grad():
        logP_total = list()
        pred_logP_total = list()
        for data in testloader:
            list_feature, list_adj, list_logP = data
            list_feature = list_feature.float()
            list_adj = list_adj.float()
            list_logP = list_logP.float()
            logP_total += list_logP.tolist()
            list_logP = list_logP.view(-1, 1)
            
            outputs = net(list_feature, list_adj)
            pred_logP_total += outputs.view(-1).tolist()

        mae = mean_absolute_error(logP_total, pred_logP_total)
        std = np.std(np.array(logP_total)-np.array(pred_logP_total))
    
    return logP_total, pred_logP_total

In [ ]:
net = GCNNet()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay = 0.00001)
criterion = nn.MSELoss()

In [ ]:
e = 300
train_losses = []
val_losses = []
for epoch in range(e):
    ts = time.time()
    train_loss = train(net=net, partition=partition, optimizer=optimizer, criterion=criterion)
    val_loss = validate(net=net, partition=partition, criterion=criterion)
    te = time.time()
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print("Epoch : {}, Loss(train/val) : {:.4f}/{:.4f}, Time : {:.2f}".format(epoch, train_loss, val_loss, te - ts))
logP_total, pred_logP_total = test(net, partition)

In [ ]:
from sklearn.linear_model import LinearRegression
logP_total = np.array(logP_total)
pred_logP_total = np.array(pred_logP_total)
model = LinearRegression()
model.fit(logP_total.reshape(-1,1), pred_logP_total.reshape(-1,1))
R2_score = model.score(logP_total.reshape(-1,1), pred_logP_total.reshape(-1,1))
coefficient = float(model.coef_)
intercept = float(model.intercept_)
print(" R2_score : {0} \n Coefficient : {1} \n Intercept : {2}".format(R2_score, coefficient, intercept))
X = np.linspace(min(logP_total),max(logP_total),100)
Y = coefficient * X + intercept

In [ ]:
list_epoch = list(range(e))
fig = plt.figure(figsize=(16,8))
ax1 = fig.add_subplot(1,2,1)
ax1.plot(list_epoch, train_losses, label = 'train_loss')
ax1.plot(list_epoch, val_losses, label = 'val_loss')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')
ax1.grid()
ax1.legend()
ax1.set_title('epoch vs loss')

ax2 = fig.add_subplot(1,2,2)
ax2.scatter(logP_total, pred_logP_total, alpha = 0.2, s = 20, c = 'b')
plt.plot(X, Y, 'c', linewidth=1 ,label=" R2_score : {0} \n Coefficient : {1} \n Intercept : {2}".format(R2_score, coefficient, intercept))
ax2.set_xlabel('true')
ax2.set_ylabel('predict')
ax2.grid()
ax2.legend()
ax2.set_title('true vs predict')
plt.show()